# Pre-Elaborazione dei Dati (Dataset di riferimento 2025 da Gennaio a Giugno)

In [10]:
import pandas as pd
import glob
import os
from sklearn.preprocessing import MinMaxScaler
import numpy as np

### Delineamo l'ambiente di lavoro

In questa sezione vengono definite le directory di lavoro e tutti quei parametri per cui andiamo a filtrare i nostri dati.

SOG_MIN --> Impostiamo il parametro a 2.0, questo ci serve per poi andare a scartare tutte le navi ferme.

TIME_GAP --> Questa è una soglia di tempo massima arbitraria permessa all'interno di una singola traiettoria. Se tra due messaggi consecutivi della stessa nave passano più di 60 minuti, assumiamo che la rotta sia stata interrotta.

In [16]:
INPUT_DIR = '../../../Dataset'
SCRIPT_DIR = os.getcwd()                                # Restituisce la directory di lavoro corrente

OUTPUT_DIR_NAME = 'Dataset_Pre-Cleaned_AIS' 
OUTPUT_DIR = os.path.join(SCRIPT_DIR, OUTPUT_DIR_NAME)

SOG_MIN_THRESHOLD = 2.0
TIME_GAP_THRESHOLD = pd.Timedelta(hours=1)

os.makedirs(OUTPUT_DIR, exist_ok=True)

all_files = glob.glob(os.path.join(INPUT_DIR, '*.parquet'))

all_clean_data = []

print(f"Trovati {len(all_files)} file Parquet da processare.")

Trovati 365 file Parquet da processare.


#### Test

- Proviamo a verificare la lettura di un file parquet e della corretta formattazione dei dati.  
- Oltre a questo andiamo ad estrarre il numero di colonne per verificare se sono state selezionate le colonne corrette.  
- Viene aggiunto anche un controllo sulle righe per vedere dopo la pulizia la percentuale di pulizia per ogni file.



In [19]:
BASE_PATH = '../../../Dataset/'

FILE_PATH_TEST = os.path.join(BASE_PATH, 'ais-2025-01-01.parquet')
FILE_PATH_TEST2 = os.path.join(BASE_PATH, 'AIS_2024_12_31.parquet')

COLUMN_MAPPING2025 = {
    'mmsi': 'MMSI', 
    'latitude': 'Latitude', 
    'longitude': 'Longitude', 
    'sog': 'SOG', 
    'cog': 'COG', 
    'base_date_time': 'Timestamp' 
}
COLUMNS_TO_READ_2025 = list(COLUMN_MAPPING2025.keys())

try:
    df = pd.read_parquet(
        FILE_PATH_TEST, 
        columns=COLUMNS_TO_READ_2025,
        engine='pyarrow' 
    )

    df = df.rename(columns=COLUMN_MAPPING2025)

    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    
    
    print(f"--- 🔍 DEBUG: Dati iniziali dal file {os.path.basename(FILE_PATH_TEST)} ---")
    print("\nHead del DataFrame:")
    print(df.head())
    print("\nTipi di Dati (Dtypes) dopo la conversione Timestamp:")
    print(df.dtypes)
    print("----------------------------------------------------------------\n")
    rows,columns = df.shape
    print(f"Numero di righe: {rows}, Numero di colonne: {columns}\n")
        
except Exception as e:
    print(f"Errore nel processare il file {FILE_PATH_TEST}: {e}")


COLUMN_MAPPING2024 = {
    'MMSI': 'MMSI',
    'LAT': 'Latitude',
    'LON': 'Longitude',
    'SOG': 'SOG',
    'COG': 'COG',
    'BaseDateTime': 'Timestamp' 
}
COLUMNS_TO_READ_2024 = list(COLUMN_MAPPING2024.keys())

try:
    df = pd.read_parquet(
        FILE_PATH_TEST2, 
        columns=COLUMNS_TO_READ_2024,
        engine='pyarrow' 
    )

    df = df.rename(columns=COLUMN_MAPPING2024)

    df['Timestamp'] = pd.to_datetime(df['Timestamp'])


    print(f"--- 🔍 DEBUG: Dati iniziali dal file {os.path.basename(FILE_PATH_TEST2)} ---")
    print("\nHead del DataFrame:")
    print(df.head())
    print("\nTipi di Dati (Dtypes) dopo la conversione Timestamp:")
    print(df.dtypes)
    print("----------------------------------------------------------------\n")
    rows,columns = df.shape
    print(f"Numero di righe: {rows}, Numero di colonne: {columns}\n")
        
except Exception as e:
    print(f"Errore nel processare il file {FILE_PATH_TEST2}: {e}")

--- 🔍 DEBUG: Dati iniziali dal file ais-2025-01-01.parquet ---

Head del DataFrame:
        MMSI  Latitude  Longitude  SOG    COG           Timestamp
0  671087100  18.46281  -66.10297  0.0  176.7 2025-01-01 00:00:00
1  367733950  48.48503 -122.60927  0.0  215.5 2025-01-01 00:00:00
2  368138010  40.47715  -73.84652  5.5  286.9 2025-01-01 00:00:02
3  367637210  29.12033  -90.21215  0.0  227.6 2025-01-01 00:00:03
4  368050000  41.27196  -72.46934  0.0  107.1 2025-01-01 00:00:03

Tipi di Dati (Dtypes) dopo la conversione Timestamp:
MMSI                  int64
Latitude            float64
Longitude           float64
SOG                 float64
COG                 float64
Timestamp    datetime64[ns]
dtype: object
----------------------------------------------------------------

Numero di righe: 7337208, Numero di colonne: 6

--- 🔍 DEBUG: Dati iniziali dal file AIS_2024_12_31.parquet ---

Head del DataFrame:
        MMSI  Latitude  Longitude   SOG    COG           Timestamp
0  367776660  21.19

### Pulizia dei dati
 
In questa sezione, iteriamo su ogni file del nostro dataset ed eseguiamo la pulizia vera e propria, applicando dei filtri. Il primo filtro filtro applicato è sulla lettura delle colonne `COLUMNS_TO_READ` prima di caricare i dati. È il modo più efficiente per scartare le colonne inutili e riduce drasticamente l'utilizzo della RAM velocizzando l'intero processo.

##### Filtri Navigazione Attiva e di Validità
  
Vengono applicati una serie di filtri per lasciare all'interno del dataset solo valori validi e di navigazione attiva:
1. Applichiamo il filtro `df = df[df['SOG'] > SOG_MIN_THRESHOLD`, eliminando i dati statici come deciso sopra.
2. Applichiamo il filtro `df[df['COG'] != 511]`,rimuovendo i record dove il COG (Course Over Ground) è $511$. Questo è un codice standard AIS che significa "Dato Non Disponibile". Senza una rotta (COG), l'informazione cinematica è incompleta e inutile per il modello.
3. Applichiamo il filtro `Filtro Lat/Lon (>= -90, <= 90, etc.)`, eliminiamo i record con coordinate geografiche errate (fuori dal globo). Questi sono errori di trasmissione o del sensore che inquinerebbero il dataset.
4. Utilizziamo il metodo `df.dopna(...)` per rimuovere qualsiasi riga che abbia valori mancanti. Questo perchè i modelli LSTM/LNN richiedono input completi per funzionare correttamente.
5. Infine l'ultimo filtro è `df['MMSI'].str.len()==9` per rimuovere i record con l'identificativo della nave non corretto. Questo perchè l'MMSI deve essere di 9 cifre e questo ci garantisce che ogni traiettoria sia attribuita ad una nave valida.

##### Segmentazione e Creazione delle Traiettorie

Questa è la fase finale prima del salvataggio, dove trasformiamo i dati puliti in sequenze coerenti (TrajectoryID).  
Quello che andiamo a fare è raggruppare i nostri dati prima per l'MMSI e poi per il TimeStamp. In questo modo abbiamo i dati ordinati ed  è possibile delineare quelle che sono le tratiettorie diverse per ogni nave. Viene aggiunta una nuova colonna al dataset che è `TrajectoryID` che ha il compito di raggruppare tutti i dati di ogni singola nave che fanno riferimento ad un intero spostamento.  
Gli spostamenti sono stati delineati assumendo che spostamenti diversi vengono caratterizzati da uno stato di navigazione non attiva di almeno 1 ora.  
Questa fase è essenziale perchè i modelli che andremo ad addestrare, impareranno non dai singoli punti ma dalle intere sequenze.

```
df = df.sort_values(by=['MMSI', 'Timestamp']).reset_index(drop=True)

            df['TimeDiff'] = df.groupby('MMSI')['Timestamp'].diff()
            df['IsNewTraj'] = (df['MMSI'] != df['MMSI'].shift(1)) | (df['TimeDiff'] > TIME_GAP_THRESHOLD)
            df['TrajectoryID'] = df['IsNewTraj'].cumsum()
```


L'ultima parte serve solo per salvare i file .parquet puliti e segmentati nel dataset che poi servirà per addestrare i modelli.

In [ ]:
MAPPING_2025 = {
    'mmsi': 'MMSI', 
    'latitude': 'Latitude', 
    'longitude': 'Longitude', 
    'sog': 'SOG', 
    'cog': 'COG', 
    'base_date_time': 'Timestamp' 
}

COLUMNS_2025 = list(MAPPING_2025.keys())

MAPPING_2024 = {
    'MMSI': 'MMSI',
    'LAT': 'Latitude',
    'LON': 'Longitude',
    'SOG': 'SOG',
    'COG': 'COG',
    'BaseDateTime': 'Timestamp'
}
COLUMNS_2024 = list(MAPPING_2024.keys())

for file_path in all_files:
    df = None
    mapping_usato = None

    try:
        
        df = pd.read_parquet(
            file_path, 
            columns=COLUMNS_2025,
            engine='pyarrow' 
        )
        df = df.rename(columns=MAPPING_2025)
        mapping_usato = "2025"
    
    except Exception as e1:
        try:
            df = pd.read_parquet(
                file_path, 
                columns=COLUMNS_2024,
                engine='pyarrow' 
            )
            df = df.rename(columns=MAPPING_2024)
            mapping_usato = "2024"
        
        except Exception as e2:
            print(f"Errore IRRISOLVIBILE nel caricare {file_path}: Schema non riconosciuto.")
            continue

    if df is not None:
        try:
            
            df['Timestamp'] = pd.to_datetime(df['Timestamp'])
            
            # Filtri cinematici e geografici
            df = df[df['SOG'] > SOG_MIN_THRESHOLD]
            df = df[df['COG'] != 511]
            df = df[(df['Latitude'] >= -90) & (df['Latitude'] <= 90)]
            df = df[(df['Longitude'] >= -180) & (df['Longitude'] <= 180)]
            
            # Filtri di integrità
            df = df.dropna(subset=['MMSI', 'Latitude', 'Longitude', 'SOG', 'COG'])
            df['MMSI'] = df['MMSI'].astype(str).str.replace(r'\D', '', regex=True)
            df = df[df['MMSI'].str.len() == 9]

            if not df.empty:

                output_filename = os.path.basename(file_path).lower()
                output_file = os.path.join(OUTPUT_DIR, output_filename)
                
                df.to_parquet(output_file, index=False)
                
                print(f"File {os.path.basename(file_path)} pre-pulito")
            
        except Exception as e:
            print(f"Errore nella FASE DI PULIZIA per il file {file_path}: {e}")

print("\n--- FASE 1 (Pre-Pulizia) completata. ---")

#### Test file pulito

In [ ]:
CLEANED_FILE_PATH_TEST = 'Dataset_Cleaned_AIS/ais-2025-01-01.parquet'

df = pd.read_parquet(
        FILE_PATH_TEST, 
        columns=COLUMNS_TO_READ_2025,
        engine='pyarrow' 
    )

df.head()
df.info()
df.describe()
print(df.dtypes)